# Open Street Map Project
### By Crystal Miller SID-000936380
___

## Map Area

Colorado Springs, CO USA

- https://www.openstreetmap.org/export#map=13/38.9326/-104.8302

Colorado Springs is an area I lived in for 15 years. I learned how to drive and had a job delivering pizzas there so am very familiar with the area. I thought it would be interesting to see how the map data compares with my memories of the area and if I can improve it in any way.
___

## Data Cleaning Process Findings
The overall data for the Colorado Springs, CO map area was very clean. After formatting and writing the data to CSV files, I used Oracle SQL to investigate the data with the following findings:
1. Validity:  
    - The sample schema given in the Udacity project directions was used. It needed to be adapted for Oracle SQL, mainly using different names for data types. (ex. TEXT vs. VARCHAR2 data types) No problems were found with data types matching the schema.
2. Accuracy:
    - Because the data is used by many people, the accuracy is subject to the users' perceptions. There are very few Map Notes in this area, indicating that it is likely to be relatively accurate.
3. Completeness:
    - When I imported the data to the database, I skipped over nodes with problem characters that might cause issues in the import process. This creates the risk that some information may be excluded unintentionally. An example of this is noted in the issues encountered section. Some phone number formats contained problem characters and were excluded. If this phone number format was used by a certain contributor, this could have caused unintentional exclusion of that contributor's data.
4. Consistency:
    - The random data points I investigated were consistent with other map sources (i.e. Google Maps)
5. Uniformity:
    - I ran queries on multiple tags, including postal codes, street types, and name direction pre/suffixes. I did not find any inconsistencies. Issues I looked for included duplicated types with different spellings and formatting. (ex. Ct vs. Court or S vs. South; see below query)
    - Formatting was consistent throughout all addresses. Phone number formatting was not consistent. Some different formats found were:
        - +1 ### ### ####
        - +1-###-###-####
        - +1 (###) ###-####
        - ###-###-####
___

### Auditing Street Types

In [ ]:
SELECT wt_value Street_Type, count(*) Occurrences
FROM ways_tags
WHERE wt_key = 'name_type'
GROUP BY wt_value
ORDER BY Occurrences DESC;

STREET_TYPE | OCCURRENCES
--- | ---
Dr       | 654  
Ct       | 299  
Rd       | 127  
Ln       | 123  
Cir      | 111  
Way      | 92  
Blvd     | 90  
Pl       | 89  
Ter      | 32  
Trl      | 32  
St       | 30  
Ave      | 15  
Loop     | 11  
Pky      | 9  
Sq       | 3  
Ln; Dr   | 3  
Dr; Ct   | 3  
Dr; Rd   | 2  
Path     | 2  
Cir; Trl | 1  
Plz      | 1  
Ln:Ln; Dr| 1  

___

## Issues Encountered in data cleaning
- csv files saved with a blank line between each record.
    - Since the files were written to csv only once, it made the most sense to correct this directly within the csv file itself.
- Phone numbers imported to csv incorrectly
    - Since the decision was made to exclude problem characters from the dataset, this caused phone numbers with certain formats to be skewed when writing to csv. If I planned on using this code on another project, this would be something that would need to be investigated and corrected.    
___

## Issues Encountered in the Map
- Multiple formats for phone numbers used
    - Pros and Cons for correcting phone number formatting:
        - Pro: Using one format would help with consistency. Inconsistent phone numbers could cause problems if this data is being used by another program.
        - Con: It would be complicated to anticipate and write code to correct each type of format. I would probably attempt to convert all of the phone numbers into a number format, removing any additional characters and then re-format them in the preferred way. This would probably involve quite a bit of trial and error.
        - Con: A relatively small number of way nodes contain phone numbers. (258 ways or 0.8%) Updating the formatting may not be worth the effort.
- Incorrect postal code encountered
    - One address contained an incorrect postal code. It has been updated in the database using the queries below.

### Phone Number Formatting
- It should be noted that the following queries were run on the dataset within the database that sampled the data without problem characters. The actual data has additional formats.

In [ ]:
SELECT count(*)
FROM ways_tags
WHERE wt_key = 'phone'
AND wt_value LIKE '+1%';

count of wt_value with '+1' at the beginning = 6

In [ ]:
SELECT count(*)
FROM ways_tags
WHERE wt_key = 'phone'
AND wt_value LIKE '%(%';

count of wt_value containing the ')' character = 6

In [ ]:
SELECT count(*)
FROM ways_tags
WHERE wt_key = 'phone'
AND wt_value LIKE '% %';

count of wt_value with at least one space = 13

In [ ]:
SELECT count(*)
FROM ways_tags
WHERE wt_key = 'phone'
AND wt_value LIKE '%-%';

count of wt_value containing the '-' character = 114

### Incorrect Postal Code
- Upon investigation of the postal codes used in the area, there were three outliers. The range excluding these is between 80907 and 80924.  Two of these are 80840 which belongs to the Air Force Academy in the area. The other is 80820 which was determined to be erroneous. 

In [ ]:
SELECT wt_value, COUNT(*)
FROM ways_tags
WHERE wt_key = 'postcode'
GROUP BY wt_value
ORDER BY wt_value;

Postcode | Occurences
--- | ---
80820	| 1
80840	| 2
80907	| 27
80917	| 12
80918	| 386
80919	| 22
80920	| 119
80923	| 459
80924	| 8

In [ ]:
SELECT wt_key "key", wt_value "value", way_user "user", way_uid "user_id"
    FROM ways_tags, ways
    WHERE ways_tags.wt_id = ways.way_id
    AND wt_id =
        (SELECT wt_id
        FROM ways_tags
        WHERE wt_value = '80820');

key | value | user| user_id
--- | --- | --- | ---
housenumber	| 1175 | b-jazz-bot	9451067
postcode |	80820 |	b-jazz-bot	9451067
street |	Chapel Hills Drive |	b-jazz-bot	9451067
amenity |	library	b-jazz-bot |	9451067
building |	yes	b-jazz-bot |	9451067
name |	Library 21C	b-jazz-bot |	9451067
operator |	Pikes Peak Library District |	b-jazz-bot |	9451067
phone |	-11402 |	b-jazz-bot |	9451067
website |	https://ppld.org/ |	b-jazz-bot |	9451067

The address, 1175 Chapel Hills Drive, shows as Pikes Peak Library District in Colorado Springs on Google Maps with the postcode 80920 instead of the value 80820 in the OSM data.<sup>2</sup> 80820 belongs to an entirely different location. (Guffey, CO) This appears to be a typo.

In [ ]:
UPDATE ways_tags
SET wt_value = '80920'
WHERE wt_value = '80820';

___

## Dataset Statistics

- Area Used
    - minlat="38.8881000" 
    - minlon="-104.9320000" 
    - maxlat="38.9772000" 
    - maxlon="-104.7284000"
- File Size
    - Colorado Springs Area OpenStreetMap OSM XML file: 53,151 KB

In [ ]:
SELECT count(UNIQUE(node_uid))
FROM nodes;

SELECT count(*)
FROM nodes;

SELECT count(*)
FROM ways;

SELECT count(*)
FROM nodes_tags
WHERE nt_value = 'school';

SELECT count(*)
FROM nodes_tags
WHERE nt_value = 'church';

Statistic | Count
--- | ---
Unique Users | 491
Nodes | 243,195
Ways | 31,303
Schools | 63
Churches | 29

## Sources

1. Github by SpecCRA https://github.com/SpecCRA/quiz_preparing_for_database_sql/blob/master/quiz_preparing_for_database_sql.py

2. Google Maps  https://www.google.com/maps/place/1175+Chapel+Hills+Dr,+Colorado+Springs,+CO+80920/@38.9534196,-104.7953066,17z/data=!3m1!4b1!4m5!3m4!1s0x87134c2a273f2c01:0xa81e962a13fd930e!8m2!3d38.9534196!4d-104.7931179
    